In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR
from MF.ALS import AlternatingLeastSquare
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender


from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

In [2]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [3]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [4]:
featureList=list(d['feature_id'])

In [5]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [6]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [7]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [8]:
urm_train_validation = urm_train + urm_validation

**Evaluation Metrics**

In [9]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [10]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [11]:
#Params for the recommenders
user_cf_param = {
    "knn": 207,
    "shrink": 2
}

item_cf_param = {
    "knn": 595,
    "shrink": 35
}

cbf_param = {
    "knn": 404,
    "shrink": 5
}

slim_bpr_param = {
    "learning_rate" : 1e-4, #0.00214168231523243
    "epochs": 10,
    "nnz" : 1,
    "knn": 1200
}

als_param = {   #work in progress...
    "n_factors": 180,
    "regularization": 1.55,
    "iterations": 15
}

Ials_param = {
    "num_factors" : 840,
    "confidence_scaling" : "linear",
    "alpha" : 0.40,
    "epsilon" : 0.28,
    "reg" : 0.0003
}

p3alpha_param = {
    "knn": 565,
    "alpha": 0.54
}

rp3beta_param = {
    "knn": 565,
    "alpha" : 0.54,
    "beta" : 0.6 
}

In [12]:
userCF = UserBasedCollaborativeFiltering(urm_train)

In [13]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

Similarity column 7947 ( 100 % ), 3313.66 column/sec, elapsed time 0.04 min


In [14]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.055300665604014684, 5007, 2940)

In [15]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [16]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

Similarity column 25975 ( 100 % ), 2611.18 column/sec, elapsed time 0.17 min


In [17]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.05125264620585148, 5007, 2940)

In [ ]:
SlimBpr=SlimBPRRec(urm_train)

In [ ]:
SlimBpr.fit(learning_rate=slim_bpr_param["learning_rate"], nnz=slim_bpr_param["nnz"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [18]:
cbf = ContentBasedFiltering(urm_train,ICM)

In [19]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

Similarity column 25975 ( 100 % ), 1263.53 column/sec, elapsed time 0.34 min


In [20]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.028753438844014722, 5007, 2940)

In [21]:
ALS = AlternatingLeastSquare(urm_train)

In [22]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [23]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.05588558969788491, 5007, 2940)

In [ ]:
IALS = IALSRecommender(urm_train)

In [ ]:
IALS.fit(num_factors=Ials_param["num_factors"], confidence_scaling=Ials_param["confidence_scaling"], alpha=Ials_param["alpha"], epsilon=Ials_param["epsilon"], reg=Ials_param["reg"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(IALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
P3alpha = P3alphaRecommender(urm_train)

In [ ]:
P3alpha.fit(topK=p3alpha_param["knn"], alpha=p3alpha_param["alpha"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(P3alpha, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
RP3beta = RP3betaRecommender(urm_train)

In [ ]:
RP3beta.fit(topK=rp3beta_param["knn"], alpha=rp3beta_param["alpha"], beta=rp3beta_param["beta"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(RP3beta, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
recommender = HybridRecommender(urm_train, ICM, userCF, itemCF, cbf, ALS)

In [ ]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"], w_p3alpha=w["p3alpha"], w_als=w["als"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

**Hybrid Recommender Tuning**

In [24]:
best_als = 0
best_MAP = 0.0
counter = 0

In [25]:
import time

w_als = 0

for i in (range (0,500)) :
    
    start = time.time()

    #Create recommender
    rec = HybridRecommender(urm_train, ICM, userCF, itemCF, cbf, ALS)
    print("Iteration " + str(counter) + " Weight w_als: " + str(w_als) +" started.")

    rec.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
            slim_param=slim_bpr_param,als_param=als_param, w_user=0.32, w_item=0.19, w_cbf=0.39, w_p3alpha=0, w_als=w_als)

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(rec, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
    print("MAP: " + str(accum_map))
    if accum_map > best_MAP :
        best_als = w_als
        print("New Best! w_als weight: " + str(w_als) +" MAP: " + str(accum_map) + " previous best MAP: " + str(best_MAP))
        best_MAP = accum_map

    stop = time.time()
    print("Execution Time: " + str(stop-start))

    w_als+=0.01




Iteration 0 Weight w_als: 0 started.
MAP: 0.06716426380627386
New Best! w_als weight: 0 MAP: 0.06716426380627386 previous best MAP: 0.0
Execution Time: 31.84040665626526
Iteration 0 Weight w_als: 0.01 started.
MAP: 0.06775452298276935
New Best! w_als weight: 0.01 MAP: 0.06775452298276935 previous best MAP: 0.06716426380627386
Execution Time: 37.52794361114502
Iteration 0 Weight w_als: 0.02 started.
MAP: 0.06761792841265428
Execution Time: 37.16857838630676
Iteration 0 Weight w_als: 0.03 started.
MAP: 0.0676715777231684
Execution Time: 42.086034059524536
Iteration 0 Weight w_als: 0.04 started.
MAP: 0.06779440503717465
New Best! w_als weight: 0.04 MAP: 0.06779440503717465 previous best MAP: 0.06775452298276935
Execution Time: 37.82295274734497
Iteration 0 Weight w_als: 0.05 started.
MAP: 0.06785778803708538
New Best! w_als weight: 0.05 MAP: 0.06785778803708538 previous best MAP: 0.06779440503717465
Execution Time: 37.66102647781372
Iteration 0 Weight w_als: 0.060000000000000005 started.


KeyboardInterrupt: 

In [ ]:
print(best_als)
print(best_MAP)

In [ ]:
w = {
    "user_cf": 0.32,
    "item_cf": 0.19, #0.3 con train
    "cbf": 0.39, #0.77
    "icm_svd": 0,
    "als": 0.14,#2
    "slim_bpr": 0,
    "elastic": 0,
    "p3alpha" : 0,
    "rp3beta" : 0
}

In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train_validation)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train_validation,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train_validation)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
P3alpha = P3alphaRecommender(urm_train_validation)

In [ ]:
P3alpha.fit(topK=p3alpha_param["knn"], alpha=p3alpha_param["alpha"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(P3alpha, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
RP3beta = RP3betaRecommender(urm_train_validation)

In [ ]:
RP3beta.fit(topK=rp3beta_param["knn"], alpha=rp3beta_param["alpha"], beta=rp3beta_param["beta"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(RP3beta, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
recommender = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS)

In [ ]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"], w_p3alpha=w["p3alpha"], w_als=w["als"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [ ]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)